# Risk Metrics Suite

## Overview
- **What this notebook does:** Demonstrates the comprehensive risk metrics suite for quantifying tail risk -- VaR, TVaR, PML, Expected Shortfall, Economic Capital, bootstrap confidence intervals, coherence testing, scenario comparison, and insurance limit selection.
- **Prerequisites:** [core/01_loss_distributions.ipynb](01_loss_distributions.ipynb)
- **Estimated runtime:** 1--2 minutes
- **Audience:** [Practitioner]

## Why Risk Metrics?
Insurance decisions hinge on understanding tail risk: the probability and severity of extreme losses. This notebook introduces a toolkit of risk metrics and shows how to use them for setting insurance limits, comparing operational scenarios, and quantifying the uncertainty around risk estimates.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from ergodic_insurance.risk_metrics import RiskMetrics, compare_risk_metrics
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# Reproducibility
np.random.seed(42)

## Configuration

In [ ]:
N_SIMULATIONS = 10_000
ANNUAL_REVENUE = 50_000_000

ATTRITIONAL_PARAMS = {
    'base_frequency': 5.0,
    'severity_mean': 50_000,
    'severity_cv': 0.8,
    'revenue_scaling_exponent': 0.3,
    'reference_revenue': 50_000_000,
}
LARGE_PARAMS = {
    'base_frequency': 0.5,
    'severity_mean': 2_000_000,
    'severity_cv': 1.2,
    'revenue_scaling_exponent': 0.2,
    'reference_revenue': 50_000_000,
}
CATASTROPHIC_PARAMS = {
    'base_frequency': 0.02,
    'severity_xm': 10_000_000,
    'severity_alpha': 2.5,
}

CONFIDENCE_LEVELS = [0.90, 0.95, 0.99, 0.995, 0.999]
RETURN_PERIODS = [10, 25, 50, 100, 200, 250, 500, 1000]

print("Risk metrics notebook configured.")

## 1. Generate Manufacturing Loss Scenarios

Simulate annual aggregate losses for a manufacturer with $50M revenue.

In [ ]:
generator = ManufacturingLossGenerator(
    attritional_params=ATTRITIONAL_PARAMS,
    large_params=LARGE_PARAMS,
    catastrophic_params=CATASTROPHIC_PARAMS,
    seed=42,
)

annual_losses = []
for _ in range(N_SIMULATIONS):
    events, year_stats = generator.generate_losses(duration=1.0, revenue=ANNUAL_REVENUE)
    annual_losses.append(sum(e.amount for e in events))

annual_losses = np.array(annual_losses)

print(f"Generated {N_SIMULATIONS:,} annual loss scenarios")
print(f"Mean annual loss: ${np.mean(annual_losses):,.0f}")
print(f"Median annual loss: ${np.median(annual_losses):,.0f}")
print(f"Max annual loss: ${np.max(annual_losses):,.0f}")

## 2. Core Risk Metrics: VaR, TVaR, and PML

- **VaR (Value at Risk):** The loss level exceeded with probability (1 - confidence)
- **TVaR (Tail VaR):** The expected loss *given* the loss exceeds VaR (captures tail severity)
- **PML (Probable Maximum Loss):** The loss expected to occur once every N years

In [ ]:
metrics = RiskMetrics(annual_losses, seed=42)

print("Value at Risk (VaR) and Tail VaR (TVaR)")
print("=" * 52)
print(f"{'Confidence':<12} {'VaR':>15} {'TVaR':>15} {'TVaR/VaR':>10}")
print("-" * 52)

var_results, tvar_results = {}, {}
for conf in CONFIDENCE_LEVELS:
    var_val = metrics.var(conf)
    tvar_val = metrics.tvar(conf)
    var_results[conf] = var_val
    tvar_results[conf] = tvar_val
    print(f"{conf:>10.1%}  ${var_val:>14,.0f}  ${tvar_val:>14,.0f}  {tvar_val / var_val:>9.2f}")

In [ ]:
print("\nProbable Maximum Loss (PML)")
print("=" * 42)
print(f"{'Return Period':<15} {'PML':>15} {'Annual Prob':>12}")
print("-" * 42)

for period in RETURN_PERIODS:
    pml_val = metrics.pml(period)
    print(f"{period:>10}-year  ${pml_val:>14,.0f}  {1 / period:>11.2%}")

## 3. Additional Metrics

Economic Capital, Maximum Drawdown, and Tail Index provide further insight into the loss distribution shape.

In [ ]:
ec_999 = metrics.economic_capital(0.999)
max_dd = metrics.maximum_drawdown()
tail_idx = metrics.tail_index()

print("Additional Risk Metrics")
print("=" * 40)
print(f"Economic Capital (99.9%): ${ec_999:,.0f}")
print(f"Maximum Drawdown: ${max_dd:,.0f}")
print(f"Tail Index (Hill estimator): {tail_idx:.2f}")
print(f"  Interpretation: {'Heavy' if tail_idx < 3 else 'Moderate'} tail")

## 4. Visualize Loss Distribution and Risk Metrics

In [ ]:
fig = metrics.plot_distribution(
    bins=50, show_metrics=True,
    confidence_levels=[0.95, 0.99, 0.995],
)
plt.suptitle('Manufacturing Loss Distribution and Risk Metrics', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 5. Bootstrap Confidence Intervals

Risk metric point estimates carry sampling uncertainty. Bootstrap confidence intervals quantify how much the estimates might shift with a different sample.

In [ ]:
print("VaR with 95% Bootstrap Confidence Intervals")
print("=" * 57)
print(f"{'Confidence':<12} {'VaR':>15} {'CI Lower':>15} {'CI Upper':>15}")
print("-" * 57)

for conf in [0.95, 0.99, 0.995]:
    result = metrics.var(conf, bootstrap_ci=True, n_bootstrap=1_000)
    ci_lower, ci_upper = result.confidence_interval
    print(f"{conf:>10.1%}  ${result.value:>14,.0f}  ${ci_lower:>14,.0f}  ${ci_upper:>14,.0f}")

## 6. Coherence Testing

A *coherent* risk measure satisfies monotonicity, sub-additivity, positive homogeneity, and translation invariance. TVaR is coherent; VaR is not. This verification helps ensure our implementation is correct.

In [ ]:
coherence_results = metrics.coherence_test()

print("Coherence Properties of TVaR")
print("=" * 40)
for prop, satisfied in coherence_results.items():
    status = "Satisfied" if satisfied else "Not satisfied"
    print(f"  {prop.replace('_', ' ').title()}: {status}")

## 7. Scenario Comparison

Compare risk metrics across a base case, a high-frequency scenario, and a high-severity scenario to understand how the loss profile changes.

In [ ]:
scenarios = {'Base Case': annual_losses}

# High frequency scenario
gen_high_freq = ManufacturingLossGenerator(
    attritional_params={**ATTRITIONAL_PARAMS, 'base_frequency': 8.0},
    large_params={**LARGE_PARAMS, 'base_frequency': 0.8},
    seed=43,
)
high_freq_losses = []
for _ in range(5_000):
    events, _ = gen_high_freq.generate_losses(duration=1.0, revenue=ANNUAL_REVENUE)
    high_freq_losses.append(sum(e.amount for e in events))
scenarios['High Frequency'] = np.array(high_freq_losses)

# High severity scenario
gen_high_sev = ManufacturingLossGenerator(
    attritional_params={**ATTRITIONAL_PARAMS, 'severity_mean': 75_000},
    large_params={**LARGE_PARAMS, 'severity_mean': 4_000_000, 'severity_cv': 1.5},
    seed=44,
)
high_sev_losses = []
for _ in range(5_000):
    events, _ = gen_high_sev.generate_losses(duration=1.0, revenue=ANNUAL_REVENUE)
    high_sev_losses.append(sum(e.amount for e in events))
scenarios['High Severity'] = np.array(high_sev_losses)

comparison_df = compare_risk_metrics(scenarios, confidence_levels=[0.95, 0.99, 0.995])
print("Risk Metrics Comparison Across Scenarios")
print("=" * 60)
print(comparison_df.round(0).to_string())

## 8. Return Period Curves and Insurance Limit Selection

Return period curves show the expected loss at each return period, directly informing insurance limit selection. Choosing a limit aligned with the 100-year PML covers all but the most extreme events.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

return_periods_plot = np.array([2, 5, 10, 25, 50, 100, 200, 500, 1000])
for name, losses in scenarios.items():
    rm = RiskMetrics(losses)
    periods, vals = rm.return_period_curve(return_periods_plot)
    ax1.semilogx(periods, vals / 1e6, 'o-', label=name, linewidth=2, markersize=6)

ax1.set_xlabel('Return Period (years)')
ax1.set_ylabel('Loss Amount ($M)')
ax1.set_title('Return Period Curves')
ax1.grid(True, alpha=0.3, which='both')
ax1.legend()

for name, losses in scenarios.items():
    sorted_losses = np.sort(losses)[::-1]
    exceedance_prob = np.arange(1, len(sorted_losses) + 1) / len(sorted_losses)
    ax2.semilogy(sorted_losses / 1e6, exceedance_prob, '-', label=name, linewidth=2)

ax2.set_xlabel('Loss Amount ($M)')
ax2.set_ylabel('Annual Exceedance Probability')
ax2.set_title('Exceedance Probability Curves')
ax2.grid(True, alpha=0.3, which='both')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
limit_options = {
    'Conservative (VaR 95%)': metrics.var(0.95),
    'Standard (VaR 99%)': metrics.var(0.99),
    'PML-100': metrics.pml(100),
    'PML-250': metrics.pml(250),
    'Comprehensive (TVaR 99%)': metrics.tvar(0.99),
}

print("Insurance Limit Selection Analysis")
print("=" * 60)
print(f"{'Limit Option':<25} {'Limit':>15} {'Coverage %':>12} {'Avg Uncovered':>15}")
print("-" * 67)

for option, limit in limit_options.items():
    covered = np.mean(annual_losses <= limit) * 100
    uncovered = annual_losses[annual_losses > limit] - limit
    avg_uncovered = np.mean(uncovered) if len(uncovered) > 0 else 0
    print(f"{option:<25} ${limit:>14,.0f} {covered:>11.1f}% ${avg_uncovered:>14,.0f}")

print("\nRecommendation:")
print(f"  Cost-conscious: VaR(95%) = ${limit_options['Conservative (VaR 95%)']:,.0f}")
print(f"  Balanced: PML-100 = ${limit_options['PML-100']:,.0f}")
print(f"  Comprehensive: TVaR(99%) = ${limit_options['Comprehensive (TVaR 99%)']:,.0f}")

## Key Takeaways

- **VaR** is intuitive but ignores tail severity; **TVaR** captures what happens beyond the threshold.
- **PML** return-period analysis directly maps to insurance limit decisions.
- Bootstrap confidence intervals quantify sampling uncertainty around point estimates.
- Coherence testing validates that TVaR behaves as a proper risk measure.
- High-severity scenarios shift the tail more than high-frequency scenarios, driving higher insurance limits.

## Next Steps

- **Study long-term dynamics:** [core/06_long_term_dynamics.ipynb](06_long_term_dynamics.ipynb)
- **Explore growth dynamics:** [core/07_growth_dynamics.ipynb](07_growth_dynamics.ipynb)
- **Optimize insurance programs:** [optimization/01_retention_optimization.ipynb](../optimization/01_retention_optimization.ipynb)